# Predicting ground states for 2D Heisenberg models

In [1]:
# Basic functionalities
import numpy as np
import random
import copy
import ast
import datetime as dt
from timeit import default_timer as timer
from os import path

# Neural tangent kernel
import jax
from neural_tangents import stax

# Traditional ML methods and techniques
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn import svm
from sklearn import linear_model
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor

In [113]:
# Load data

length = 5 # length = 4, 5, 6, 7, 8, 9 for orig; only 4, 5, 6, 7 for new
width = 5

shadow_size = 50 # up to 1000

Xfull = [] # Shape = (number of data) x (number of params)
Ytrain = [] # Shape = (number of data) x (number of pairs), estimated 2-point correlation functions
Yfull = [] # Shape = (number of data) x (number of pairs), exact 2-point correlation functions

def get_path_prefix(data='orig'):
    prefix = './heisenberg_data/heisenberg_{}x{}'.format(length, width)
    if data == 'new':
        prefix = './new_data/data_{}x{}/simulation_{}x{}'.format(length, width, length, width)
    return prefix
    
data_name = 'orig'
prefix = get_path_prefix(data=data_name)

for idx in range(1, 301):
    if path.exists('{}_id{}_XX.txt'.format(prefix, idx)) == False:
        continue
    with open('{}_id{}_samples.txt'.format(prefix, idx), 'r') as f:
        single_data = []
        classical_shadow_big = [[int(c) for i, c in enumerate(line.split("\t"))] for line in f]
        classical_shadow = classical_shadow_big[0:shadow_size]
        for i in range(length * 5):
            for j in range(length * 5):
                if i == j:
                    single_data.append(1.0)
                    continue
                corr = 0
                cnt = 0
                for shadow in classical_shadow:
                    if shadow[i] // 2 == shadow[j] // 2:
                        corr += 3 if shadow[i] % 2 == shadow[j] % 2 else -3
                    cnt += 1
                single_data.append(corr / cnt)
        Ytrain.append(single_data)
    with open('{}_id{}_XX.txt'.format(prefix, idx), 'r') as f:
        single_data = []
        for line in f:
            for i, c in enumerate(line.split("\t")):
                v = float(c)
                single_data.append(v)
        Yfull.append(single_data)
    with open('{}_id{}_couplings.txt'.format(prefix, idx), 'r') as f:
        single_data = []
        for line in f:
            for i, c in enumerate(line.split("\t")):
                v = float(c)
                single_data.append(v)
        Xfull.append(single_data)
        
        
# Print information
Xfull = np.array(Xfull)
print("number of data (N) * number of params (m) =", Xfull.shape)
Ytrain = np.array(Ytrain)
Yfull = np.array(Yfull)
print("number of data (N) * number of pairs =", Yfull.shape)

# Normalize Xfull
xmin = np.amin(Xfull)
xmax = np.amax(Xfull)

# normalize so that all entries are between -1 and 1 using min-max feature scaling
Xfull_norm = np.array(list(map(lambda row : list(map(lambda x : -1 + 2*(x - xmin)/(xmax - xmin), row)), Xfull)))

# grid of qubits
grid = np.array(range(1, length * width + 1)).reshape((length, width))

# generate all edges in grid in same order as Xfull
all_edges = []
for i in range(0, length):
    for j in range(1, width + 1):
        if i != length - 1:
            all_edges.append((width * i + j, width * (i + 1) + j))
        if j != width:
            all_edges.append((width * i + j, width * i + j + 1))
print(all_edges)

number of data (N) * number of params (m) = (100, 40)
number of data (N) * number of pairs = (100, 625)
[(1, 6), (1, 2), (2, 7), (2, 3), (3, 8), (3, 4), (4, 9), (4, 5), (5, 10), (6, 11), (6, 7), (7, 12), (7, 8), (8, 13), (8, 9), (9, 14), (9, 10), (10, 15), (11, 16), (11, 12), (12, 17), (12, 13), (13, 18), (13, 14), (14, 19), (14, 15), (15, 20), (16, 21), (16, 17), (17, 22), (17, 18), (18, 23), (18, 19), (19, 24), (19, 20), (20, 25), (21, 22), (22, 23), (23, 24), (24, 25)]


In [19]:
# Categorizing pairs of qubits by distance
            
def calc_distance(q1, q2):
    # Given two qubits q1, q2 (1-indexed integers) in length x width grid
    # Output l1 distance between q1 and q2 in grid

    pos1 = np.array(np.where(grid == q1)).T[0]
    pos2 = np.array(np.where(grid == q2)).T[0]

    return np.abs(pos1[0] - pos2[0]) + np.abs(pos1[1] - pos2[1])

def get_nearby_qubit_pairs(d):
    # Given distance d > 0
    # Output all pairs of qubits that are within distance d of each other
    
    if d == 1:
        return all_edges
    
    qubit_pairs = []
    for q1 in range(1, length * width + 1):
        for q2 in range(1, length * width + 1):
            dist = calc_distance(q1, q2)
            pair = tuple(sorted((q1, q2)))
            if dist == d and pair not in qubit_pairs:
                qubit_pairs.append(pair)
    
    return qubit_pairs
            

[(1, 6), (1, 2), (2, 7), (2, 3), (3, 8), (3, 4), (4, 9), (4, 5), (5, 10), (6, 11), (6, 7), (7, 12), (7, 8), (8, 13), (8, 9), (9, 14), (9, 10), (10, 15), (11, 16), (11, 12), (12, 17), (12, 13), (13, 18), (13, 14), (14, 19), (14, 15), (15, 20), (16, 17), (17, 18), (18, 19), (19, 20)]


In [6]:
# Finding local patches of a given radius

def get_local_region_qubits(q, delta1):
    # Given a qubit q (1-indexed integer) in length x width grid and radius delta1
    # delta1 = -1 if all qubits are in local region
    # Output list of qubits (1-indexed integers) within a radius of delta1 of q
    
    if delta1 == 0:
        return [q]
    elif delta1 == -1:
        return list(range(1, length * width + 1))
    
    local_qubits = []
    for q2 in range(1, length * width + 1):
        dist = calc_distance(q, q2)
        
        if dist <= delta1:
            local_qubits.append(q2)
    
    return local_qubits

def get_local_region_edges(q1, q2, delta1):
    # Given two qubits q1, q2 (1-indexed integers) in length x width grid and radius delta1
    # delta1 = -1 if all qubits are in local region
    # Output list of tuples of qubits (1-indexed integers) corresponding to edges in local region of radius delta1

    if delta1 == 0:
        return [(q1, q2)]
    elif delta1 == -1:
        return all_edges

    local_qubits = list(set(get_local_region_qubits(q1, delta1) + get_local_region_qubits(q2, delta1)))
    
    local_edges = []
    for edge in all_edges:
        (q1, q2) = edge
        if q1 in local_qubits and q2 in local_qubits:
            local_edges.append(edge)

    return local_edges

def get_local_region_params(q1, q2, delta1, data, i):
    # Given two qubits q1, q2 (1-indexed integers) in length x width grid, radius delta1, and input data (i.e., Xfull)
    # delta1 = -1 if all qubits are considered nearby
    # Output data but only for parameters corresponding to edges within radius delta1
    
    edges = get_local_region_edges(q1, q2, delta1)
    
    indices = [all_edges.index(edge) for edge in edges]
    
    return np.array([data[i][j] for j in sorted(indices)])
    

In [7]:
print('edges: ' + str(get_local_region_edges(1,2,1)))
print('params: ' + str(get_local_region_params(1,2,1, Xfull_norm, 0)))

edges: [(1, 6), (1, 2), (2, 7), (2, 3), (6, 7)]
params: [ 0.18222392  0.53439556  0.13297216 -0.07949298  0.29838656]


In [8]:
# Feature mapping

def get_feature_vectors(delta1, R, data, omega, gamma=1.0, q1=0, q2=1):
    # Given radius delta1 and hyperparameter R (number of nonlinear features per local region), input data, and fixed randomness omega
    # delta1 = -1 if all qubits are considered nearby
    # Output concatenated feature vectors
    
    # to store all concatenated feature vectors
    all_feature_vectors = []
    
    for i in range(len(data)):
        feature_vector_concat = []
        # iterate over all possible local regions
        n = len(all_edges)
        for k in range(n):
            (q1, q2) = all_edges[k]
            data_local = get_local_region_params(q1, q2, delta1, data, i)
            m_local = len(data_local)

            # do nonlinear feature map on each vector in data_local
            feature_vector = []

            for j in range(R):
                omega_j = omega[k][j]
                val = np.exp(np.dot(omega_j, data_local) * gamma / (m_local ** 0.5) * 1j)
                feature_vector.append(np.real(val))
                feature_vector.append(np.imag(val))

            # concatenate feature vectors together
            feature_vector_concat += feature_vector
            
        all_feature_vectors.append(feature_vector_concat)
        
    # note all_feature_vectors is of size number of data (N) x (2 * R * number of local regions)
    return np.array(all_feature_vectors)
        

In [115]:
# Training and testing algorithm

# set size of local region
delta1 = 0

# set max number of feature entries
max_R = 1000

# set of pairs of qubits we care about predicting correlation function for
d = 2
qubits = get_nearby_qubit_pairs(d)
print(qubits)

# set test size
test_size = 0.5

train_idx, test_idx, _, _ = train_test_split(range(len(Xfull)), range(len(Xfull)), test_size=test_size, random_state=0)

# generate omega to pass into feature mapping
omega = []
for (q1, q2) in all_edges:
    m_local = len(get_local_region_edges(q1, q2, delta1))
    omega_sub = []
    for j in range(max_R):
        omega_sub.append(np.random.normal(0, 1, m_local))
    omega.append(omega_sub)

data_path = './clean_results/new_algorithm/test_size={}_shadow_size={}_qubits_d={}'.format(test_size, shadow_size, d)
with open('{}/results_{}x{}_{}_data.txt'.format(data_path, length, width, data_name), 'w') as f1, open('{}/coefficients_{}x{}_{}_data.txt'.format(data_path, length, width, data_name), 'w') as f2:
    for (q1, q2) in qubits:
        print('(q1, q2) =', (q1, q2))
        print('(q1, q2) =', (q1, q2), file=f1)
        print('(q1, q2) =', (q1, q2), file=f2)

        def train_and_predict():
            # consider the pair (q1, q2)
            global q1, q2

            # training data (estimated from measurement data)
            y = np.array([Ytrain[i].reshape((length * width, length * width))[q1 - 1][q2 - 1] for i in range(len(Xfull))])
            X_train, X_test, y_train, y_test = train_test_split(Xfull_norm, y, test_size=test_size, random_state=0)

            # testing data (exact expectation values)
            y_clean = np.array([Yfull[i].reshape((length * width, length * width))[q1 - 1][q2 - 1] for i in range(len(Xfull))])
            _, _, _, y_test_clean = train_test_split(Xfull_norm, y_clean, test_size=test_size, random_state=0)

            # use cross validation to find the best hyperparameters
            best_cv_score, test_score = 999.0, 999.0
            ML_method = lambda Cx : linear_model.Lasso(alpha=Cx, max_iter=10000)
            best_coef = []
            #ML_method = lambda Cx: KernelRidge(kernel='linear', alpha=Cx)

            for R in [5, 10, 20, 40]:
                for gamma in [0.4, 0.5, 0.6, 0.65, 0.7, 0.75]:
                    # feature mapping
                    Xfeature_train = get_feature_vectors(delta1, R, X_train, omega, gamma, q1, q2)
                    Xfeature_test = get_feature_vectors(delta1, R, X_test, omega, gamma, q1, q2)

                    for alpha in [2**(-8), 2**(-7), 2**(-6), 2**(-5)]:
                        score = -np.mean(cross_val_score(ML_method(alpha), Xfeature_train, y_train, cv=4, scoring="neg_root_mean_squared_error"))
                        if best_cv_score > score:
                            clf = ML_method(alpha).fit(Xfeature_train, y_train.ravel())
                            test_score = np.linalg.norm(clf.predict(Xfeature_test).ravel() - y_test_clean.ravel()) / (len(y_test) ** 0.5)
                            best_cv_score = score
                            best_coef = clf.coef_.reshape((len(all_edges), 2 * R))
        
                            #coef = clf.coef_.reshape((len(all_edges), 2 * R))
                            #print(list(zip(all_edges, np.linalg.norm(coef, axis=1))))
                            #print(R, gamma, alpha, score, test_score)

            coef_edges = list(zip(all_edges, np.linalg.norm(best_coef, axis=1)))
            return best_cv_score, test_score, coef_edges

        print(train_and_predict()[0:2], file=f1)
        print(train_and_predict()[2], file=f2)
        print(train_and_predict())

[(1, 3), (1, 7), (1, 11), (2, 4), (2, 6), (2, 8), (2, 12), (3, 5), (3, 7), (3, 9), (3, 13), (4, 8), (4, 10), (4, 14), (5, 9), (5, 15), (6, 8), (6, 12), (6, 16), (7, 9), (7, 11), (7, 13), (7, 17), (8, 10), (8, 12), (8, 14), (8, 18), (9, 13), (9, 15), (9, 19), (10, 14), (10, 20), (11, 13), (11, 17), (11, 21), (12, 14), (12, 16), (12, 18), (12, 22), (13, 15), (13, 17), (13, 19), (13, 23), (14, 18), (14, 20), (14, 24), (15, 19), (15, 25), (16, 18), (16, 22), (17, 19), (17, 21), (17, 23), (18, 20), (18, 22), (18, 24), (19, 23), (19, 25), (20, 24), (21, 23), (22, 24), (23, 25)]
(q1, q2) = (1, 3)
(0.26434322472676197, 0.16304830062318318, [((1, 6), 0.0), ((1, 2), 0.0), ((2, 7), 0.0), ((2, 3), 0.0), ((3, 8), 0.05613344029838536), ((3, 4), 0.0), ((4, 9), 0.0), ((4, 5), 0.0), ((5, 10), 0.0), ((6, 11), 0.07432405856689961), ((6, 7), 0.0), ((7, 12), 0.0), ((7, 8), 0.15002250967937608), ((8, 13), 0.026011563123217004), ((8, 9), 0.0820291086472333), ((9, 14), 0.04697590435993424), ((9, 10), 0.0), ((

(0.23071225646799173, 0.05978553604296986, [((1, 6), 0.0), ((1, 2), 0.0), ((2, 7), 0.0), ((2, 3), 0.0), ((3, 8), 0.0), ((3, 4), 0.0), ((4, 9), 0.0), ((4, 5), 0.0), ((5, 10), 0.0), ((6, 11), 0.0), ((6, 7), 0.0), ((7, 12), 0.0), ((7, 8), 0.0), ((8, 13), 0.016451976398541374), ((8, 9), 0.0), ((9, 14), 0.0), ((9, 10), 0.0), ((10, 15), 0.0), ((11, 16), 0.0), ((11, 12), 0.0), ((12, 17), 0.0), ((12, 13), 0.0), ((13, 18), 0.0), ((13, 14), 0.0), ((14, 19), 0.0), ((14, 15), 0.0), ((15, 20), 0.0), ((16, 21), 0.0), ((16, 17), 0.0), ((17, 22), 0.0), ((17, 18), 0.0), ((18, 23), 0.0), ((18, 19), 0.0), ((19, 24), 0.0), ((19, 20), 0.0), ((20, 25), 0.0), ((21, 22), 0.0), ((22, 23), 0.0), ((23, 24), 0.0), ((24, 25), 0.0)])
(q1, q2) = (4, 8)
(0.24226401907390316, 0.09364841998230036, [((1, 6), 0.0), ((1, 2), 0.0), ((2, 7), 0.0), ((2, 3), 0.0), ((3, 8), 0.0), ((3, 4), 0.0), ((4, 9), 0.0), ((4, 5), 0.0), ((5, 10), 0.0), ((6, 11), 0.0), ((6, 7), 0.0), ((7, 12), 0.0), ((7, 8), 0.0), ((8, 13), 0.0), ((8, 9), 0

(0.2193185519693248, 0.05419269058956752, [((1, 6), 0.0), ((1, 2), 0.0), ((2, 7), 0.0), ((2, 3), 0.0), ((3, 8), 0.0213208483026535), ((3, 4), 0.0), ((4, 9), 0.0), ((4, 5), 0.00905116745035281), ((5, 10), 0.0), ((6, 11), 0.0), ((6, 7), 0.0), ((7, 12), 0.0), ((7, 8), 0.0), ((8, 13), 0.0), ((8, 9), 0.0), ((9, 14), 0.0), ((9, 10), 0.0), ((10, 15), 0.0), ((11, 16), 0.0), ((11, 12), 0.0), ((12, 17), 0.0068284795979343706), ((12, 13), 0.0), ((13, 18), 0.0), ((13, 14), 0.0), ((14, 19), 0.0), ((14, 15), 0.0), ((15, 20), 0.006696718516732984), ((16, 21), 0.0), ((16, 17), 0.0), ((17, 22), 0.0), ((17, 18), 0.0), ((18, 23), 0.0), ((18, 19), 0.0), ((19, 24), 0.0), ((19, 20), 0.0), ((20, 25), 0.0), ((21, 22), 0.0), ((22, 23), 0.0), ((23, 24), 0.0), ((24, 25), 0.0)])
(q1, q2) = (7, 17)
(0.2797634435804457, 0.05414055159531677, [((1, 6), 0.0), ((1, 2), 0.0), ((2, 7), 0.0), ((2, 3), 0.0), ((3, 8), 0.0), ((3, 4), 0.0), ((4, 9), 0.0), ((4, 5), 0.0), ((5, 10), 0.0), ((6, 11), 0.0), ((6, 7), 0.0), ((7, 12),

(0.22791911056480318, 0.08365472268482108, [((1, 6), 0.0), ((1, 2), 0.0), ((2, 7), 0.0), ((2, 3), 0.0), ((3, 8), 0.009897624521608668), ((3, 4), 0.0), ((4, 9), 0.0), ((4, 5), 0.0), ((5, 10), 0.0), ((6, 11), 0.0), ((6, 7), 0.0), ((7, 12), 0.0), ((7, 8), 0.0), ((8, 13), 0.0), ((8, 9), 0.0), ((9, 14), 0.0), ((9, 10), 0.0), ((10, 15), 0.0), ((11, 16), 0.0), ((11, 12), 0.0), ((12, 17), 0.0), ((12, 13), 0.0), ((13, 18), 0.0), ((13, 14), 0.0), ((14, 19), 0.0), ((14, 15), 0.0), ((15, 20), 0.0), ((16, 21), 0.0), ((16, 17), 0.05263965185729902), ((17, 22), 0.0), ((17, 18), 0.0), ((18, 23), 0.0), ((18, 19), 0.0), ((19, 24), 0.026160440061191514), ((19, 20), 0.0), ((20, 25), 0.0), ((21, 22), 0.0), ((22, 23), 0.0), ((23, 24), 0.040349305295094395), ((24, 25), 0.0)])
(q1, q2) = (11, 17)
(0.2681833035290173, 0.06959981545253408, [((1, 6), 0.0), ((1, 2), 0.0), ((2, 7), 0.0), ((2, 3), 0.0), ((3, 8), 0.0), ((3, 4), 0.0), ((4, 9), 0.0), ((4, 5), 0.0), ((5, 10), 0.0), ((6, 11), 0.0), ((6, 7), 0.0), ((7, 1

(0.262969860502783, 0.08893252680996254, [((1, 6), 0.0), ((1, 2), 0.0), ((2, 7), 0.0), ((2, 3), 0.0), ((3, 8), 0.0), ((3, 4), 0.0), ((4, 9), 0.0), ((4, 5), 0.0), ((5, 10), 0.0), ((6, 11), 0.0), ((6, 7), 0.0), ((7, 12), 0.0), ((7, 8), 0.0), ((8, 13), 0.0), ((8, 9), 0.0), ((9, 14), 0.0), ((9, 10), 0.0), ((10, 15), 0.0), ((11, 16), 0.0), ((11, 12), 0.0), ((12, 17), 0.0), ((12, 13), 0.0), ((13, 18), 0.0), ((13, 14), 0.0), ((14, 19), 0.0), ((14, 15), 0.0), ((15, 20), 0.0), ((16, 21), 0.0), ((16, 17), 0.0), ((17, 22), 0.0), ((17, 18), 0.0), ((18, 23), 0.0), ((18, 19), 0.0), ((19, 24), 0.0), ((19, 20), 0.0), ((20, 25), 0.0), ((21, 22), 0.0), ((22, 23), 0.0), ((23, 24), 0.0), ((24, 25), 0.0)])
(q1, q2) = (14, 20)
(0.24136392665515166, 0.08891343701699603, [((1, 6), 0.0), ((1, 2), 0.02279238611242156), ((2, 7), 0.014488396413238921), ((2, 3), 0.0), ((3, 8), 0.0), ((3, 4), 0.0), ((4, 9), 0.04068503919276195), ((4, 5), 0.0), ((5, 10), 0.14630267561641086), ((6, 11), 0.0), ((6, 7), 0.0), ((7, 12),

(0.1970254776551184, 0.14273860709986824, [((1, 6), 0.0), ((1, 2), 0.075412999491553), ((2, 7), 0.06752528477622192), ((2, 3), 0.06849659731144331), ((3, 8), 0.024017612486336406), ((3, 4), 0.0), ((4, 9), 0.06952983330791435), ((4, 5), 0.14299426063981216), ((5, 10), 0.0), ((6, 11), 0.14177320319300987), ((6, 7), 0.05648391274182617), ((7, 12), 0.0), ((7, 8), 0.0), ((8, 13), 0.11449196224154472), ((8, 9), 0.0), ((9, 14), 0.0), ((9, 10), 0.21053077158953407), ((10, 15), 0.12352085464402549), ((11, 16), 0.00549135351235372), ((11, 12), 0.05829058253129668), ((12, 17), 0.05493291486080227), ((12, 13), 0.14411415776495523), ((13, 18), 0.04699631939823344), ((13, 14), 0.0024185910852679146), ((14, 19), 0.0023619591615249374), ((14, 15), 0.053926715583745635), ((15, 20), 0.0), ((16, 21), 0.0), ((16, 17), 0.0), ((17, 22), 0.0), ((17, 18), 0.15902446139352328), ((18, 23), 0.0695500001959109), ((18, 19), 0.0), ((19, 24), 0.044032036097168735), ((19, 20), 0.0), ((20, 25), 0.030309430443815205), 

In [111]:
# Training and testing algorithm
# Running with linear regression

# set size of local region
delta1 = 0

# set max number of feature entries
max_R = 1000

# set of pairs of qubits we care about predicting correlation function for
d = 3
qubits = get_nearby_qubit_pairs(d)
print(qubits)

# set test size
test_size = 0.1

train_idx, test_idx, _, _ = train_test_split(range(len(Xfull)), range(len(Xfull)), test_size=test_size, random_state=0)

# generate omega to pass into feature mapping
omega = []
for (q1, q2) in all_edges:
    m_local = len(get_local_region_edges(q1, q2, delta1))
    omega_sub = []
    for j in range(max_R):
        omega_sub.append(np.random.normal(0, 1, m_local))
    omega.append(omega_sub)

data_path = './clean_results/test_size={}_shadow_size={}_{}_data_qubits_d={}_linear_regression'.format(test_size, shadow_size, data_name, d)
with open('{}/results_{}x{}.txt'.format(data_path, length, width), 'w') as f1, open('{}/coefficients_{}x{}.txt'.format(data_path, length, width), 'w') as f2:
    for (q1, q2) in qubits:
        print('(q1, q2) =', (q1, q2))
        print('(q1, q2) =', (q1, q2), file=f1)
        print('(q1, q2) =', (q1, q2), file=f2)

        def train_and_predict():
            # consider the pair (q1, q2)
            global q1, q2

            # training data (estimated from measurement data)
            y = np.array([Ytrain[i].reshape((length * width, length * width))[q1 - 1][q2 - 1] for i in range(len(Xfull))])
            X_train, X_test, y_train, y_test = train_test_split(Xfull_norm, y, test_size=test_size, random_state=0)

            # testing data (exact expectation values)
            y_clean = np.array([Yfull[i].reshape((length * width, length * width))[q1 - 1][q2 - 1] for i in range(len(Xfull))])
            _, _, _, y_test_clean = train_test_split(Xfull_norm, y_clean, test_size=test_size, random_state=0)

            # use cross validation to find the best hyperparameters
            best_cv_score, test_score = 999.0, 999.0
            #ML_method = lambda Cx : linear_model.Lasso(alpha=Cx, max_iter=10000)
            best_coef = []
            ML_method = linear_model.LinearRegression()
            #ML_method = lambda Cx: KernelRidge(kernel='linear', alpha=Cx)

            for R in [5, 10, 20, 40]:
                for gamma in [0.4, 0.5, 0.6, 0.65, 0.7, 0.75]:
                    # feature mapping
                    Xfeature_train = get_feature_vectors(delta1, R, X_train, omega, gamma, q1, q2)
                    Xfeature_test = get_feature_vectors(delta1, R, X_test, omega, gamma, q1, q2)

                    score = -np.mean(cross_val_score(ML_method, Xfeature_train, y_train, cv=4, scoring="neg_root_mean_squared_error"))
                    if best_cv_score > score:
                        clf = ML_method.fit(Xfeature_train, y_train.ravel())
                        test_score = np.linalg.norm(clf.predict(Xfeature_test).ravel() - y_test_clean.ravel()) / (len(y_test) ** 0.5)
                        best_cv_score = score
                        best_coef = clf.coef_.reshape((len(all_edges), 2 * R))

                        #coef = clf.coef_.reshape((len(all_edges), 2 * R))
                        #print(list(zip(all_edges, np.linalg.norm(coef, axis=1))))
                        #print(R, gamma, alpha, score, test_score)

            coef_edges = list(zip(all_edges, np.linalg.norm(best_coef, axis=1)))
            return best_cv_score, test_score, coef_edges

        print(train_and_predict()[0:2], file=f1)
        print(train_and_predict()[2], file=f2)
        print(train_and_predict())

[(1, 4), (1, 8), (1, 12), (1, 16), (2, 5), (2, 9), (2, 11), (2, 13), (2, 17), (3, 6), (3, 10), (3, 12), (3, 14), (3, 18), (4, 7), (4, 13), (4, 15), (4, 19), (5, 8), (5, 14), (5, 20), (6, 9), (6, 13), (6, 17), (7, 10), (7, 14), (7, 16), (7, 18), (8, 11), (8, 15), (8, 17), (8, 19), (9, 12), (9, 18), (9, 20), (10, 13), (10, 19), (11, 14), (11, 18), (12, 15), (12, 19), (13, 16), (13, 20), (14, 17), (15, 18), (16, 19), (17, 20)]


FileNotFoundError: [Errno 2] No such file or directory: './clean_results/test_size=0.1_shadow_size=1000_orig_data_qubits_d=3_linear_regression/results_4x5.txt'

In [200]:
#
# Dirichlet kernel
#

kernel_dir = np.zeros((len(Xfull), Xfull.shape[1]*5))
for i, x1 in enumerate(Xfull):
    cnt = 0
    for k in range(len(x1)):
        for k1 in range(-2, 3):
            kernel_dir[i, cnt] += np.cos(np.pi * k1 * x1[k])
            cnt += 1
print("constructed Dirichlet kernel")
            
#
# Neural tangent kernel
#
    
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(1)
)
kernel_NN2 = kernel_fn(Xfull, Xfull, 'ntk')

init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(1)
)
kernel_NN3 = kernel_fn(Xfull, Xfull, 'ntk')
                
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(1)
)
kernel_NN4 = kernel_fn(Xfull, Xfull, 'ntk')

init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(1)
)
kernel_NN5 = kernel_fn(Xfull, Xfull, 'ntk')

list_kernel_NN = [kernel_NN2, kernel_NN3, kernel_NN4, kernel_NN5]

for r in range(len(list_kernel_NN)):
    kernel = list_kernel_NN[r].copy()
    for i in range(len(list_kernel_NN[r])):
        for j in range(len(list_kernel_NN[r])):
            # list_kernel_NN[r][i][j] /= (list_kernel_NN[r][i][i] * list_kernel_NN[r][j][j]) ** 0.5
            list_kernel_NN[r][i].at[j].divide((kernel[i][i] * kernel[j][j]) ** 0.5)
print("constructed neural tangent kernel")
            
#
# RBF kernel is defined in Sklearn
#
print("RBF kernel (will be constructed in sklearn)")


constructed Dirichlet kernel
constructed neural tangent kernel
RBF kernel (will be constructed in sklearn)


In [201]:
# Training and testing algorithm (Old method)

# set of pairs of qubits we care about predicting correlation function for
d = 1
qubits = get_nearby_qubit_pairs(d)

test_size = 0.4

train_idx, test_idx, _, _ = train_test_split(range(len(Xfull)), range(len(Xfull)), test_size=test_size, random_state=0)

#with open('./results/orig_algorithm/orig_algorithm_test_size={}/orig_algorithm_{}_data_k/results_{}x{}_all_qubits.txt'.format(test_size, data_name, length, width), 'w') as f:
for (q1, q2) in qubits[7:8]:
    # each k corresponds to the correlation function in a pair of qubits
    print("(q1, q2) = ({}, {})".format(q1, q2))
    #print("k =", k, file=f)

    def train_and_predict(kernel, opt="linear"): # opt="linear" or "rbf"

        # instance-wise normalization
        for i in range(len(kernel)):
            if type(kernel) == np.ndarray:
                kernel[i] /= np.linalg.norm(kernel[i])
            else:
                kernel.at[i].divide(np.linalg.norm(kernel[i]))

        # consider the k-th pair
        global q1, q2

        # training data (estimated from measurement data)
        y = np.array([Ytrain[i].reshape((length * width, length * width))[q1 - 1][q2 - 1] for i in range(len(Xfull))])
        X_train, X_test, y_train, y_test = train_test_split(kernel, y, test_size=test_size, random_state=0)

        # testing data (exact expectation values)
        y_clean = np.array([Yfull[i].reshape((length * width, length * width))[q1 - 1][q2 - 1] for i in range(len(Xfull))])
        _, _, _, y_test_clean = train_test_split(kernel, y_clean, test_size=test_size, random_state=0)

        # use cross validation to find the best method + hyper-param
        best_cv_score, test_score = 999.0, 999.0
        for ML_method in [(lambda Cx: svm.SVR(kernel=opt, C=Cx)), (lambda Cx: KernelRidge(kernel=opt, alpha=1/(2*Cx)))]:
            for C in [0.0125, 0.025, 0.05, 0.125, 0.25, 0.5, 1.0, 2.0]:
                score = -np.mean(cross_val_score(ML_method(C), X_train, y_train, cv=5, scoring="neg_root_mean_squared_error"))
                if best_cv_score > score:
                    clf = ML_method(C).fit(X_train, y_train.ravel())
                    test_score = np.linalg.norm(clf.predict(X_test).ravel() - y_test_clean.ravel()) / (len(y_test) ** 0.5)
                    best_cv_score = score

        return best_cv_score, test_score

    # Dirichlet
    #print("Dirich. kernel", train_and_predict(kernel_dir), file=f)
    print("Dirich. kernel", train_and_predict(kernel_dir))
    # RBF
    #print("Gaussi. kernel", train_and_predict(Xfull, opt="rbf"), file=f)
    print("Gaussi. kernel", train_and_predict(Xfull, opt="rbf"))
    # Neural tangent
    for kernel_NN in list_kernel_NN:
        #print("Neur. T kernel", train_and_predict(kernel_NN), file=f)
        print("Neur. T kernel", train_and_predict(kernel_NN))

(q1, q2) = (4, 5)
Dirich. kernel (0.2524725471445278, 0.24720161876210964)
Gaussi. kernel (0.18580648811215386, 0.19334888028240627)
Neur. T kernel (0.13558781374889525, 0.1640358656018234)
Neur. T kernel (0.1397908436522027, 0.16704989523840308)
Neur. T kernel (0.16193101833276144, 0.18731811705888335)
Neur. T kernel (0.19950551325783114, 0.21130932478677994)
